In [59]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
import pycountry as pc

In [60]:
#Lista con los paises
countries = [country.name for country in pc.countries]

In [61]:
colaboraciones_2019 = pd.read_csv('colaboraciones_with_names_2019.csv')
colaboraciones_2019 = colaboraciones_2019.drop_duplicates()
colaboraciones_2020 = pd.read_csv('colaboraciones_with_names_2020.csv')
colaboraciones_2020 = colaboraciones_2020.drop_duplicates()
colaboraciones_2021 = pd.read_csv('colaboraciones_with_names_2021.csv')
colaboraciones_2021 = colaboraciones_2021.drop_duplicates()
colaboraciones_2022 = pd.read_csv('colaboraciones_with_names_2022.csv')
colaboraciones_2022 = colaboraciones_2022.drop_duplicates()
print(len(colaboraciones_2019))
print(len(colaboraciones_2020))
print(len(colaboraciones_2021))
print(len(colaboraciones_2022))
colaboraciones_2019.head()

840
1572
1236
1698


,doi,author1,name1,aff1,author2,name2,aff2
0,http://dx.doi.org/10.1109/5gwf.2019.8911693,https://dblp.org/pid/75/4085,Mérouane Debbah,"Khalifa University, Abu Dhabi, United Arab Emi...",https://dblp.org/pid/119/7827,Miguel Ángel Vázquez,Centre Tecnològic de Telecomunicacions de Cata...
1,https://doi.org/10.1109/5gwf.2019.8911693,https://dblp.org/pid/75/4085,Mérouane Debbah,"Khalifa University, Abu Dhabi, United Arab Emi...",https://dblp.org/pid/119/7827,Miguel Ángel Vázquez,Centre Tecnològic de Telecomunicacions de Cata...
2,http://dx.doi.org/10.1007/978-3-030-21074-8_26,https://dblp.org/pid/l/MongLiLee,Mong-Li Lee,National University of Singapore (NUS),https://dblp.org/pid/243/1972-1,Quang Duc Nguyen 0001,"Singapore Eye Research Institute, Singapore"
3,https://doi.org/10.1007/978-3-030-21074-8_26,https://dblp.org/pid/l/MongLiLee,Mong-Li Lee,National University of Singapore (NUS),https://dblp.org/pid/243/1972-1,Quang Duc Nguyen 0001,"Singapore Eye Research Institute, Singapore"
4,http://dx.doi.org/10.1007/978-3-030-21074-8_26,https://dblp.org/pid/l/MongLiLee,Mong-Li Lee,National University of Singapore (NUS),https://dblp.org/pid/h/WynneHsu,Wynne Hsu,National University of Singapore (NUS)


In [62]:
#Agregamos el año a cada dataframe
colaboraciones_2019['year'] = 2019
colaboraciones_2020['year'] = 2020
colaboraciones_2021['year'] = 2021
colaboraciones_2022['year'] = 2022

In [63]:
universidades = pd.read_csv('cwurData.csv')
universidades_and_country = universidades[['institution', 'country']]
universidades_and_country = universidades_and_country.drop_duplicates()

In [102]:
import networkx as nx
from pyvis.network import Network
import matplotlib.pyplot as plt

pares_autores_2019 = colaboraciones_2019[['name1', 'aff1','name2', 'aff2', 'doi']].drop_duplicates()
pares_autores_2019['pairs'] = pares_autores_2019['name1'] + ' - ' + pares_autores_2019['name2']

# Create an empty graph
G = nx.Graph()

# Add edges connecting name1 with name2
for index, row in pares_autores_2019.iterrows():
    name1 = row['name1']
    name2 = row['name2']
    G.add_edge(name1, name2)

# Create a pyvis network
nt = Network(notebook=True)

# Add nodes and edges to the network
for node in G.nodes:
    nt.add_node(node)

for edge in G.edges:
    nt.add_edge(edge[0], edge[1])

# Display the network
nt.show('graph_2019.html')


graph_2019.html


In [96]:
pares_autores_2020 = colaboraciones_2020[['name1', 'aff1','name2', 'aff2', 'doi']].drop_duplicates()
pares_autores_2020['pairs'] = pares_autores_2020['name1'] + ' - ' + pares_autores_2019['name2']

# Create an empty graph
G = nx.Graph()

# Add edges connecting name1 with name2
for index, row in pares_autores_2020.iterrows():
    name1 = row['name1']
    name2 = row['name2']
    G.add_edge(name1, name2)

# Create a pyvis network
nt = Network(notebook=True)

# Add nodes and edges to the network
for node in G.nodes:
    nt.add_node(node)

for edge in G.edges:
    nt.add_edge(edge[0], edge[1])

# Display the network
nt.show('graph_2020.html')

graph_2020.html


In [97]:
pares_autores_2021 = colaboraciones_2021[['name1', 'aff1','name2', 'aff2', 'doi']].drop_duplicates()
pares_autores_2021['pairs'] = pares_autores_2021['name1'] + ' - ' + pares_autores_2019['name2']

# Create an empty graph
G = nx.Graph()

# Add edges connecting name1 with name2
for index, row in pares_autores_2021.iterrows():
    name1 = row['name1']
    name2 = row['name2']
    G.add_edge(name1, name2)

# Create a pyvis network
nt = Network(notebook=True)

# Add nodes and edges to the network
for node in G.nodes:
    nt.add_node(node)

for edge in G.edges:
    nt.add_edge(edge[0], edge[1])

# Display the network
nt.show('graph_2021.html')

graph_2021.html


In [98]:
pares_autores_2022 = colaboraciones_2022[['name1', 'aff1','name2', 'aff2', 'doi']].drop_duplicates()
pares_autores_2022['pairs'] = pares_autores_2022['name1'] + ' - ' + pares_autores_2019['name2']

# Create an empty graph
G = nx.Graph()

# Add edges connecting name1 with name2
for index, row in pares_autores_2022.iterrows():
    name1 = row['name1']
    name2 = row['name2']
    G.add_edge(name1, name2)

# Create a pyvis network
nt = Network(notebook=True)

# Add nodes and edges to the network
for node in G.nodes:
    nt.add_node(node)

for edge in G.edges:
    nt.add_edge(edge[0], edge[1])

# Display the network
nt.show('graph_2022.html')

graph_2022.html


In [107]:
count_df_2019 = pares_autores_2019[pares_autores_2019['aff1'] != pares_autores_2019['aff2']].groupby('name1').size().reset_index(name='count')
count_df_2020 = pares_autores_2020[pares_autores_2020['aff1'] != pares_autores_2020['aff2']].groupby('name1').size().reset_index(name='count')
count_df_2021 = pares_autores_2021[pares_autores_2021['aff1'] != pares_autores_2021['aff2']].groupby('name1').size().reset_index(name='count')
count_df_2022 = pares_autores_2022[pares_autores_2022['aff1'] != pares_autores_2022['aff2']].groupby('name1').size().reset_index(name='count')

count_all_2019 = pares_autores_2019.groupby('name1').size().reset_index(name='count')
count_all_2020 = pares_autores_2020.groupby('name1').size().reset_index(name='count')
count_all_2021 = pares_autores_2021.groupby('name1').size().reset_index(name='count')
count_all_2022 = pares_autores_2022.groupby('name1').size().reset_index(name='count')